In [1]:
%%writefile C:\Users\Cflhxb\AppData\Local\anaconda3\Lib\APSIMOptimiserTools.py

import os
import xmltodict, json
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import MathsUtilities as MUte

BlankManager = {'$type': 'Models.Manager, Models',
            'CodeArray': [],
            'Parameters': None,
            'Name': 'SetCropParams',
            'IncludeInDocumentation': False,
            'Enabled': True,
            'ReadOnly': False}

SetCropParams = {
          "$type": "Models.Manager, Models",
          "CodeArray": [],
          "Parameters": [],
          "Name": "SetCropParams",
          "IncludeInDocumentation": False,
          "Enabled": True,
          "ReadOnly": False}

CodeArrayStart = [
            "using System;",
            "using Models.Core;",
            "using Models.Interfaces;",
            "using Models.PMF;",
            "using APSIM.Shared.Utilities;",
            "",
            "namespace Models",
            "{",
            "    [Serializable]",
            "    public class Script : Model",
            "    {",
            "        [Link] Zone zone;",
            "        [EventSubscribe(\"Sowing\")]",
            "        private void OnSowing(object sender, EventArgs e)",
            "        {",
            "            object Pval = 0;"
             ]
            
    
CodeArrayEnd = [
            "        }       ",
            "    }",
            "}"
          ]

# def AppendModeltoModelofTypeAndDeleteOldIfPresent(Parent,TypeToAppendTo,ModelToAppend):
#     #try:
#     for child in Parent['Children']:
#         if child['$type'] == TypeToAppendTo:
#             pos = 0
#             for g in child['Children']:
#                 if g['Name'] == ModelToAppend['Name']:
#                     del child['Children'][pos]
#                     #print('Model ' + ModelToAppend['Name'] + ' found and deleted')
#                 pos+=1
#             child['Children'].append(ModelToAppend)
#         else:
#             Parent = AppendModeltoModelofTypeAndDeleteOldIfPresent(child,TypeToAppendTo,ModelToAppend)
#     return Parent
#     #except:
#     #    return Parent
    
def AppendModeltoModelofType(Parent,TypeToAppendTo,ModelToAppend,NameToAppendTo = ""):
    try:
        for child in Parent['Children']: #Assess each child of the parent model
            if (child['$type'] == TypeToAppendTo): #If they are of the right type check their name
                if ((child['Name'] == NameToAppendTo) | (NameToAppendTo == "")): #if the name is right or no name arguement given proceed
                    pos = 0
                    for kid in child['Children']: #Check all the kids of the model we are going to append to 
                        if ((kid['$type'] == ModelToAppend['$type']) & (kid['Name'] == ModelToAppend['Name'])): #if there is already one of the same type and name as the one we will append
                            del child['Children'][pos]  # delete it
                            break # and exit loop
                        else:
                            pos+=1  # increment positoin of child and assess the next one
             
                    child['Children'].append(ModelToAppend) #Add model to the appropriate location
                    #print('Model ' + ModelToAppend['Name'] + ' added to ' +Name)
                    break # exit loop
                else:
                    Parent = AppendModeltoModelofType(child,TypeToAppendTo,ModelToAppend,NameToAppendTo) #check the children
            else:
                Parent = AppendModeltoModelofType(child,TypeToAppendTo,ModelToAppend,NameToAppendTo) #check the children
        return Parent
    except:
        return Parent
    
def RemoveModelofType(Parent,TypeToRemove,NameToRemove):
    pos = 0 
    for child in Parent['Children']: #Assess each child of the parent model
        try:
            if len(child['Children'])>0:
                RemoveModelofType(child,TypeToRemove,NameToRemove)
            if ((child['Name'] == NameToRemove) & (child['$type'] == TypeToRemove)): #if the name and type match
                del Parent['Children'][pos]  # delete it
        except:
            return Parent
        pos +=1
    return Parent
    
def findNextChild(Parent,ChildName):
    if len(Parent['Children']) >0:
        for child in range(len(Parent['Children'])):
            if Parent['Children'][child]['Name'] == ChildName:
                return Parent['Children'][child]
    else:
        return Parent[ChildName]

def findModel(Parent,PathElements):
    for pe in PathElements:
        #print(pe)
        Parent = findNextChild(Parent,pe)
    return Parent   

def replaceModel(Parent,modelPath,New):
    PathElements = modelPath.split('.')
    if PathElements[-1][-1] != "]":
        
        findModel(Parent,PathElements[:-1])[PathElements[-1]] = New
    else:
        findModel(Parent,PathElements[:-1])[PathElements[-1][0]][int(PathElements[-1][-2])-1] = New

def StopReporting(Apsimx,modelPath):
    PathElements = modelPath.split('.')
    report = findModel(Apsimx,PathElements)
    report["EventNames"] = []

def removeModel(Parent,modelPath):
    PathElements = modelPath.split('.')
    Parent = findModel(Parent,PathElements[:-1])
    pos = 0
    found = False
    for c in Parent['Children']:
        if c['Name'] == PathElements[-1]:
            del Parent['Children'][pos]
            found = True
            break
        pos += 1
    if found == False:
        print('Failed to find ' + PathElements[-1] + ' to delete')

def ApplyParamReplacementSet(paramValues,paramNames,filePath):
    with open(filePath,'r') as ApsimxJSON:
        Apsimx = json.load(ApsimxJSON)
        ApsimxJSON.close()
    ## Remove old prameterSet manager in replacements
    removeModel(Apsimx,'Replacements.SetCropParams')

    ## Add crop coefficient overwrite into replacements
    codeArray = CodeArrayStart.copy()
    for p in range(len(paramValues)):
        codeArray.append("            Pval = " + str(paramValues[p]) + ";")
        codeArray.append("            zone.Set(\"" + paramNames[p] + "\"" + ", Pval);")
    for p in CodeArrayEnd:
        codeArray.append(p)
    
    SetCropParams["CodeArray"] = codeArray

    AppendModeltoModelofType(Apsimx,"Models.Core.Folder, Models",SetCropParams,"Replacements")

    with open(filePath,'w') as ApsimxJSON:
        json.dump(Apsimx,ApsimxJSON,indent=2)
        
def makeLongString(SimulationSet):
    longString =  '/SimulationNameRegexPattern:"'
    longString =  longString + '(' + SimulationSet[0]  + ')|' # Add first on on twice as apsim doesn't run the first in the list
    for sim in SimulationSet[:]:
        longString = longString + '(' + sim + ')|'
    longString = longString + '(' + SimulationSet[-1] + ')'#|' ## Add Last on on twice as apsim doesnt run the last in the list
    #longString = longString + '(' + SimulationSet[-1] + ')"'
    return longString

def CalcScaledValue(Value,RMax,RMin):
    return (Value - RMin)/(RMax-RMin)

def CartesianParameterCombinations(paramNames,rangeAndNumber):
    ParamRanges = pd.DataFrame(index = paramNames, 
                               columns = ['Lower','Upper','Number'],
                               data = rangeAndNumber)        
    ParamRanges.loc[:,'Values'] = ''
    for p in ParamRanges.index:
        ParamRanges.at[p,'Values'] = np.linspace(ParamRanges.loc[p,'Lower'],ParamRanges.loc[p,'Upper'],ParamRanges.loc[p,'Number'])

    combinationsCartesian = pd.MultiIndex.from_product(ParamRanges.loc[:,'Values'].values)
    combinationsList = []
    for c in range(len(combinationsCartesian)):
        combinationsList.append(list(combinationsCartesian[c]))
    ParameterCombinations = pd.DataFrame(columns=paramNames,data=combinationsList)  
    return ParameterCombinations

#Make sure there is a blank "SetCropParams" manager in each simulation
def AddBlankSetCropParams(APSIMXfilePath):
    with open(APSIMXfilePath,'r') as ApsimxJSON:
        Apsimx = json.load(ApsimxJSON)
        ApsimxJSON.close()

    AppendModeltoModelofType(Apsimx,"Models.Core.Zone, Models",SetCropParams)

    with open(APSIMXfilePath,'w') as ApsimxJSON:
        json.dump(Apsimx,ApsimxJSON,indent=2)
        
#Make sure there is a blank "SetCropParams" manager in each simulation
def RemoveBlankSetCropParams(APSIMXfilePath):
    with open(APSIMXfilePath,'r') as ApsimxJSON:
        Apsimx = json.load(ApsimxJSON)
        ApsimxJSON.close()
        RemoveModelofType(Apsimx,"Models.Manager, Models","SetCropParams")
    with open(APSIMXfilePath,'w') as ApsimxJSON:
        json.dump(Apsimx,ApsimxJSON,indent=2)
        
def PlotResults(obsPreSet,ret,ParamCombs,FittingVariables,ShortParams):
    graph = plt.figure(figsize=(10,20))
    gs = gridspec.GridSpec(6, 6)
    ax = graph.add_subplot(gs[0, 0:2])
    PlotObsPre(obsPreSet,ax,FittingVariables,ax)

    ax = graph.add_subplot(gs[0, 4:6])
    CurrentBest = [-ParamCombs.loc[0,'NSE']]
    for x in ParamCombs.index[1:]:
        CurrentBest.append(max(-ParamCombs.loc[x,'NSE'],CurrentBest[x-1]))
    plt.plot(CurrentBest,'-',color='b')
    plt.ylabel('Best NSE achieved')
    plt.xlabel('Itteration')
    plt.ylim(-2,1)
    bestFit = ParamCombs.loc[:,'NSE'].idxmin()
    adequateFits = min(sum(ParamCombs.sort_values('NSE').NSE.values<-0.5),sum(ParamCombs.sort_values('NSE').NSE.values<ParamCombs.loc[bestFit,'NSE']*.9))
    try:
        CandidateSets = ParamCombs.sort_values('NSE').iloc[:adequateFits,:].copy()
        bestSetInd = CandidateSets.loc[:,'CombScore'].idxmax()
        BestSet = CandidateSets.loc[bestSetInd,:] 
    except:
        BestSet = ParamCombs.loc[bestFit,:]
        print('No acceptable fits')
    pos = 0
    for p in ShortParams.index:
        ax = graph.add_subplot(gs[1,pos])
        plt.plot(ParamCombs.loc[:,p],-ParamCombs.loc[:,'NSE'],'o',color='k')
        #plt.plot(x0[pos],max(0,-ret.func_vals[0]),'o',color='orange',ms=8,mec='k',mew=2)
        plt.plot(ParamCombs.loc[bestFit,p],-ParamCombs.loc[bestFit,'NSE'],'o',color='cyan',ms=8,mec='k',mew=2)
        plt.plot(BestSet[p],-BestSet['NSE'],'o', ms=8, mfc="None",mec='m', mew=2)
        if pos == 0:
            plt.ylabel('NSE')
        else:
            ax.axes.yaxis.set_visible(False)
        plt.xlabel(ShortParams[p])
        plt.ylim(0,1)
        
        ax = graph.add_subplot(gs[2,pos])
        plt.plot(ParamCombs.sort_values('NSE').loc[:,p].values,'o',ms=1)
        plt.plot([0,adequateFits],[ParamCombs.loc[bestFit,p]]*2,'--',color='cyan',ms=8,mec='k',mew=2,lw=1)
        plt.plot([0],[ParamCombs.loc[bestFit,p]],'-',color='cyan',ms=4,mec='k',mew=2)
        initRank = ParamCombs.loc[0,'Rank']
        plt.plot(BestSet['Rank'],BestSet[p],'o', ms=8, mfc="None",mec='m', mew=2)
        if pos == 0:
            plt.ylabel('Parameter value')
        plt.xlabel('Rank')
        ax.tick_params(axis='y',direction='out', pad=1,rotation=90)
        ax2 = ax.twinx()
        ax2.plot(-ParamCombs.sort_values('NSE').loc[:,'NSE'].values ,'-',color='r')
        ax.set_xscale('log')
        plt.fill_between([adequateFits,len(ParamCombs.NSE.values)],1,-3,color='r',alpha=0.1)
        plt.ylim(0,1)
        if pos == 5:
            plt.ylabel('NSE')
        else:
            ax2.axes.yaxis.set_ticklabels([])
        pos+=1
        
def PlotObsPre(obsPreSet,ax,FittingVariables,leg=True):
    indloc = 0
    plotScObsPred(obsPreSet,1,FittingVariables)
    plt.plot([0,1],[0,1],'-',color='k')
    plt.ylabel('Scalled Predictions')
    plt.xlabel('Scalled Observations')
    if leg==True:
        plt.legend(bbox_to_anchor=(1.1, 1.05))
    RegStats = MUte.MathUtilities.CalcRegressionStats('LN',obsPreSet.loc[:,'ScPred'].values,obsPreSet.loc[:,'ScObs'].values)
    plt.text(1.6,0.03,'new NSE = '+str(RegStats.NSE)[:4],transform=ax.transAxes,horizontalalignment='right')
    #plt.text(0.03,0.97,c,horizontalalignment='left')

CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
    
def plotScObsPred(obsPreSet,alp,FittingVariables,leg=True):
    markers = ['o','s','^','v','1','2','3','4','+','x','d']
    colors = pd.Series(index = FittingVariables,
                       data = CB_color_cycle[:len(FittingVariables)])
    
    obsPredSet = obsPreSet.sort_values(['Var','Predicted.Experiment']).copy()
    for var in obsPreSet.loc[:,'Var'].drop_duplicates():
        epos = 0
        if leg==True:
            lablab = var.replace('Wheat.Phenology.','')
        else:
            lablab = None
        for expt in obsPreSet.loc[:,'Predicted.Experiment'].drop_duplicates():
            filt = ((obsPreSet.loc[:,'Var'] == var) & (obsPreSet.loc[:,'Predicted.Experiment'] == expt))
            plt.plot(obsPreSet.loc[filt,'ScObs'],obsPreSet.loc[filt,'ScPred'],
                     markers[epos],color=colors[var],alpha=alp,label=lablab)
            
            epos+=1
            if epos == 11:
                epos= 0
            lablab = None

Overwriting C:\Users\Cflhxb\AppData\Local\anaconda3\Lib\APSIMOptimiserTools.py
